In [29]:
import pandas as pd
import json
import plotly.plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
offline.init_notebook_mode(connected=True)

In [4]:

with open('characters.json', 'r') as f:
    content = f.readlines()
    processed_data = [inner_content for line in content for inner_content in json.loads(line)['results']]
    

In [5]:
characters_data = []
for data in processed_data:
    fil_data = {}
    fil_data['name'] = data['name']
    fil_data['no_of_comics'] = data['comics']['available']
    fil_data['no_of_events'] = data['events']['available']
    fil_data['no_of_stories'] = data['stories']['available']
    fil_data['no_of_series'] = data['series']['available']
    characters_data.append(fil_data)

In [6]:
charc_df = pd.DataFrame(characters_data)
charc_df.head()

,name,no_of_comics,no_of_events,no_of_stories,no_of_series
0,3-D Man,12,1,21,3
1,A-Bomb (HAS),3,0,7,2
2,A.I.M.,49,0,52,33
3,Aaron Stack,14,0,27,3
4,Abomination (Emil Blonsky),53,1,63,26


In [10]:
top_15_by_no_of_comics = charc_df.sort_values('no_of_comics', ascending=False).iloc[:15]

    

In [13]:
buddies = go.Bar(
    x = top_15_by_no_of_comics['no_of_comics'],
    y = top_15_by_no_of_comics['name'],
    name = 'Top 15 by No of Comics',
    orientation='h',
    text=top_15_by_no_of_comics['no_of_comics'],
    textposition='outside'
)

data = [buddies]
layout = go.Layout(
    barmode = 'group',
    bargroupgap=0.1,
    width = 900,
    yaxis=dict(autorange="reversed")
)

fig = go.Figure(data = data, layout = layout)
fig.update_yaxes(automargin=True)
offline.iplot(fig)

In [30]:
from datetime import datetime

In [31]:
with open('events.json', 'r') as f:
    content = f.readlines()
    series_list = [inner_content for line in content for inner_content in json.loads(line)['results']]

In [33]:
events_filtered_data = []
for indx,data in enumerate(series_list):
    fil_data = {}
    fil_data['id']=data['id']
    fil_data['title'] = data['title']
    fil_data['start_date'] = datetime.strptime(data['start'], '%Y-%m-%d %H:%M:%S') if data['start'] else None
    fil_data['end_date'] = datetime.strptime(data['end'],'%Y-%m-%d %H:%M:%S') if data['end'] and datetime.strptime(data['end'],'%Y-%m-%d %H:%M:%S') < datetime.today() else datetime.today()
    fil_data['stories'] = data['stories']['available']
    fil_data['characters'] = data['characters']['available']
    fil_data['index']=indx
    events_filtered_data.append(fil_data)


In [34]:
events_df = pd.DataFrame(events_filtered_data)

In [35]:

events_df['duration'] = events_df['end_date'] - events_df['start_date']

In [36]:
top_10_events = events_df.sort_values('characters', ascending=False).iloc[:10]

In [38]:

buddies = go.Bar(
    x = top_10_events['characters'],
    y = top_10_events['title'],
    name = 'Top 10',
    orientation='h',
    text=top_10_events['characters'],
    textposition='outside'
)

data = [buddies]
layout = go.Layout(
    barmode = 'group',
    bargroupgap=0.1,
    width = 900,
    xaxis = {
        'title': 'Number of Characters'
    },
    
    yaxis={
        'autorange': "reversed",
        'title': "Name of the event"
    }
)

fig = go.Figure(data = data, layout = layout)
fig.update_yaxes(automargin=True)
offline.iplot(fig)

In [24]:
longest_running_events = events_df.sort_values('duration', ascending=False).iloc[:10]

In [25]:
import numpy as np
import math
longest_running_events['duration'] = longest_running_events['duration'] /  np.timedelta64(1,'Y')


In [26]:
longest_running_events['duration'] = longest_running_events['duration'].astype(int)


In [27]:

buddies = go.Bar(
    x = longest_running_events['duration'],
    y = longest_running_events['title'],
    name = 'Top 10',
    orientation='h',
    text=longest_running_events['duration'],
    textposition='outside'
)

data = [buddies]
layout = go.Layout(
    barmode = 'group',
    bargroupgap=0.1,
    width = 900,
    xaxis = {
        'title': "Running time in years"
    },
    yaxis=dict(autorange="reversed", title="Name of the event")
)

fig = go.Figure(data = data, layout = layout)
fig.update_yaxes(automargin=True)
offline.iplot(fig)

In [34]:
longest_running_events.loc[: ,['title', 'start_date', 'end_date']]

,title,start_date,end_date
41,Secret Wars,1984-05-10,2008-12-17
63,Kraven's Last Hunt,1987-10-10,2012-03-21
43,Secret Wars II,1985-07-10,2009-05-13
21,Mutant Massacre,1986-10-10,2010-01-06
7,Armor Wars,1987-12-10,2010-03-17
8,Atlantis Attacks,1989-01-01,2011-02-16
34,Inferno,1988-12-10,2010-08-25
29,Fall of the Mutants,1988-01-10,2007-01-17
0,Acts of Vengeance!,1989-12-10,2008-01-04
56,Infinity Gauntlet,1991-07-10,2005-01-19
